<table align="left" style="border-style: hidden" class="table"> <tr><td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon256.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Fall 2018</h4><p>Ani Adhikari and Jim Pitman</p>CC BY-NC 4.0</div></td></tr></table><!-- not in pdf -->

In [ ]:
from datascience import *
from prob140 import *
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
%matplotlib inline
import numpy as np
from scipy import stats
import ipywidgets as widgets
from ipywidgets import interact

# Lab 10: Chinese Restaurant Process, Part I #

In this lab you will analyze a stochastic model for *clustering*, which is a process of visualizing and organizing data based on similarities between sampled individuals. Methods of identifying clusters have applications in a wide variety of areas. Here are just a few examples.

- Criminology: locations where a type of crime occurs
- Marketing: customers who share preferences
- Biology: populations segmented by patterns in genes
- Natural Language Processing: words grouped by similarity of meaning or grammatical properties

One commonly used model is *k-means clustering*, which assumes that the data are continuous numerical variables and that the number of clusters in the population is known to be $k$. By contrast, the *Chinese Restuarant Process* is a discrete time stochastic process that can be used as a model for clustering and does not assume a fixed number of clusters. Instead, the clusters evolve randomly as individuals enter, according to a specified probabilistic structure. The data could be categorical or numerical.

The Chinese Restaurant Process is an example of a *generative Bayesian model* which specifies the probabilstic rules by which the process evolves and in which the representation itself evolves as more data come in.

**History:** The process has its origins in the work of [Warren Ewens](https://en.wikipedia.org/wiki/Warren_Ewens) in the early 1970's, in particular the [Ewens Sampling Formula](https://en.wikipedia.org/wiki/Ewens%27s_sampling_formula) of population genetics. Since then, the development of the theory of the stochastic process and its use in machine learning has been very much a Berkeley enterprise: Jim Pitman, David Aldous, and Mike Jordan are among the people involved. The restaurant analogy is due to Jim Pitman and our late colleague Lester Dubins, during one of their regular cafe meetings decades ago. Though there is a popular story that they came up with it while eating at a Chinese restaurant, in fact they were at the Strada on Bancroft and College. 

**What you will learn:**
We have distributed the analysis across two labs so that you can reasonably complete it and have time to think about the ideas. In each lab, part of the analysis will be by simulation and part by probability calculations. The first lab involves only discrete distributions. The second lab will contain long run analysis involving connections made in class between the beta and binomial distributions.

In Prob 140 we will focus on understanding the probability model. You can then go on to other classes that discuss how to fit the model to data.

This lab is about the distribution of the total number of clusters observed in a fixed amount of time. This theory helps answer questions like these, about $n$ observations:
- How many different animal species do you expect to see?
- What is the distribution of the number of different types of documents that you will have?
- What is the chance that everybody is retweeting the same tweet?

The "restaurant" image is that each person entering the restaurant chooses a table to join. Thus each table becomes a cluster. The model allows for an infinite number of tables, each of infinite size, though of course if you make $n$ observations then the observed number of occupied tables will be finite. That's the number of different clusters observed.

In this lab you will study the random number of observed clusters. You will simulate a Chinese Restaurant process and find the empirical distribution of the number of occupied tables. You will also find the mean and SD of the number of occupied tables, both empirically and analytically.

## The Process ##
In keeping with the Chinese Restaurant analogy, think of clusters as groups of people sitting at the same table. We will only consider occupied tables, so the number of tables is equal to the number of clusters. 

The process evolves according to the following rules.
- There is a positive parameter $\theta$.
- People enter the restaurant one at a time.
- Person 1 enters and sits at a table which we will call Table 1.
- Each subsequent person 
    - either joins an existing table with probability proportional to the number of people already at that table, or
    - starts a new table with probability proportional to $\theta$.
- People choose tables independently of each other.

Don't worry about running out of room. The restaurant has infinite capacity and each table is infinitely large. You can imagine infinitely many such tables at the start, or imagine new tables appearing magically each time a person's random choice is to start a new table. We prefer the second image because it consists only of the occupied tables.

Note that the tables are not numbered at the start. We number them according to their order of formation. Thus Table 1 is the table at which Person 1 sits. Table 2 is the next new table to be formed. We can't say exactly who starts it, because that's random. Table 3 is the third new table to be formed. And so on.

To visualiaze the process, run the cell below. It simulates 100 people arriving according to a Chinese Restaurant process with some value of $\theta$. Move the slider slowly at first, so that you can see the people coming in one at a time. 

Run the cell several times. The value of $\theta$ changes each time, and you will see quite a bit of variation in the results. This indicates that you might be able to use this model in varied applications, by setting the parameter appropriately.

In [ ]:
visualize_cr()

In Parts 1 and 2 below, you will write the code to simulate the Chinese Restaurant process. In Parts 3 through 5 you will study the distribution of the total number of occupied tables after $N$ people have been seated. In other words, you will study the distribution of the number of clusters produced by this process when it has run for $N$ steps.

#newpage

## Part 1: Steps towards Simulation ##
Start by running the cell below to set the parameter $\theta$.

In [ ]:
theta = 1

The goal is to create an array `people` that shows the number of people at each table. If there are 10 people, of whom 6 are at Table 1, 3 at Table 2, and 1 at Table 3, then the array created should be [6, 3, 1]. 

All entries in the array must be positive. The length of the array will be the total number of tables formed, which is random.

The cell below starts off the array `people` when the first person enters at sits at Table 1. Run the cell. 

In [ ]:
people = make_array(1)

To help code the selection of the next person who enters, we will also keep track of the labels of the tables formed, in the array `tables`. That will be an array of consecutive integers starting at 1. If `people` is [6, 3, 1] then `tables` is [1, 2, 3]. If `people` is [10] then `tables` is [1]. 

Run the cell below to create `tables` when just one person is in the system.

In [ ]:
tables = make_array(1)

To write the code below it might help you to keep the array [6, 3, 1] in mind as an example of what `people` might look like after 10 people are seated.

### a) ###
How can you use `people` to determine how many people are already seated when a new person comes in? Assign this to `n` in the cell below. Your code should work not just for the "starter" array `people` created above, but for any `people` array that shows the number of people at each table.

In [ ]:

# The expression should involve the array named people

n = ...

Your process hasn't started evolving yet so at this stage what should the value of `n` be? Run the cell below to check that it gives the right answer. If it doesn't, start by resetting `tables` and `people` above and then fix your calculation of `n`. 

In [ ]:
n

### b) ###
Use `tables` to create an array that contains the choices of tables that the next newly entering person will have, by filling out the cell below. There are no probabilities involved, yet. Just create an array of choices, that is, all the possible tables at which the new person could sit.

In [ ]:
# The label of the new table, if this person starts a new table
new_table = ...

# Array of all possible table choices that this new person has:
tbl_choices = ...

### c) ###
The array `tbl_choices` contains all the possible tables the new person can select. Create an array `tbl_probs` containing the corresponding probabilities of selection. That is, the $i$th element of `tbl_probs` should be the probability that the new person selects the $i$th element of tbl_choices.

To figure out what to calculate, you will need to scroll back to the description of the model and work out the probabilities by hand on scratch paper.

In [ ]:
# Array of probabilities of selecting the different tables
tbl_probs = ...

Check that `tbl_probs` is a probability distribution, by running the cell below.

In [ ]:
sum(tbl_probs)

### d) ###
Now make the new person's choice of table, and assign it to the name `choice`. Use `np.random.choice` to do so. The call `np.random.choice(values, p=probabilities)` makes one random draw from the distribution that has possible values in the array `values` and the corresponding probabilities in the array `probabilities`.

One technicality: You have to force the choice to be of the `int` type, because later you will have to use it as an array index.

In [ ]:
# The random choice made by the new person

choice = int(...)

At this stage, your process is just beginning to evolve: you started with one person, and now a new person has made their choice. What could the possible values of `choice` be at this stage? 

Run the cell below to confirm that `choice` is one of these possible values.

In [ ]:
choice

### e) ###
Write code (as many lines as you need) that updates the arrays `tables` and `people` appropriately based on the random choice made by the new person in Part (d). Your code should work for a new person entering at any stage.

Ask yourself the following questions before writing your code:
- Under what circumstances should `tables` be updated, and how should it be updated?
- Under what circumstances should `people` be updated, and how should it be updated?

Your code can use any of the quantities calculated in earlier cells: `theta`, `people`, `tables`, `new_table`, `tbl_choices`, `tbl_probs`, `choice`.

If you need to change the element at index `k` of an array `my_array`, say by multiplying the original element by 2, you can use `my_array[k] = my_array[k] * 2`.

After you run the cell, both `tables` and `people` should be consistent with `choice`. If you need to trouble-shoot, remember to Run All Above first, so that all the variables get reset.

Run the cell below to check that your code gives consistent answers for the choice made by Person 2 in Part **d**.

If it doesn't, remember to Run All Above to reset before you fix your calculation.

In [ ]:
# Person 2's choice of table

choice, tables, people

#newpage

## Part 2: Code the Simulation ##

### a) ###
Collect the code in Part 1 and use it to define a function `cr` that takes `N` and `theta` as its arguments, runs the Chinese Restaurant Process till `N` people have been seated, and returns an array of the number of people at each table in the order of table formation.

- If you call your function with the arguments 1 and any positive $\theta$, it should return the array [1]. That represents the one person seated at Table 1.
- If you call your function with the arguments 2 and any positive $\theta$, it should return either the array [2] if both people sit at Table 1, or [1, 1] if Person 2 starts a new table.
- If you call your function with the arguments 3 and any positive $\theta$, then it should return one of the following arrays: [3], [2, 1], [1, 2], [1, 1, 1].
- And so on.

### b) ###
Perform a basic ballpark check that your function is working right: run the cell below and confirm that the expression is evaluating to the right answer.

In [ ]:
sum(cr(1000, 1))

#newpage

## Part 3: Run the Process ##

### a) ###
Run the process with $\theta = 1$ and 100 people. Display the simulated results a table that has two columns:
- `Table`: The labels of the occupied tables, in order of formation, so that the first entry is 1 and the last entry is the number of tables at the end of the run.
- `People at Table`: The number of people seated at the corresponding table.

Run the cell several times to get a sense of the variability of the results.

In [ ]:
N = 100
theta = 1

...
...
simulated_process = Table().with_column(
    'Table', ...,
    'People at Table', ...
)

simulated_process.show()

### b) ### 

Now, pass your results into `visualize_cr` to visualize the process.

In [ ]:
visualize_cr(cr(N, theta))

### c) ###
Repeat Part (b) for varying values of $N$ and $\theta$. Be sure to try $N = 100, 1000$, and $10000$ with $\theta = 0.5, 1,$ and $2$. Keep your eye on the number of tables as well as the distribution of people across tables.

### d) ###
Give a brief qualitative description of what you have seen in the simulations for $N = 100$ and $\theta = 0.5, 1$, and $2$. Address questions such as:
- Are there lots of tables, not many, or is it not possible to tell?
- Is the distribution of the number of people pretty uniform across the tables? If not, describe what you see about the number of big and small clusters.
- In what way does $\theta$ make a difference? Or is it not possible to tell?


**Type your answer here**

#newpage

## Part 4: Empirical Distribution of the Number of Tables ##

### a) ###

Define a function `num_tables` that takes `N`, `theta`, and `repetitions` (the number of times to simulate the process) as its arguments. In each repetition, the function runs the Chinese Restaurant Process with `N` people and counts the number of tables occupied by those people once they are all seated. The function should return an array of length `repetitions` consisting of the simulated table counts.

In [ ]:

def num_tables(N, theta, repetitions):
    ...
    ...
    return tbls

### b) ###
Draw the empirical histogram of the number of tables, in the case $N = 100$ and $\theta = 1$, based on 2000 repetitions. Be prepared to wait as the simulation chugs.

The code below uses the `prob140` method `emp_dist`, which takes an array of integer data and plots its histogram. It saves you the trouble of turning the array into a table and calling `hist` with the appropriate bins.

In [ ]:

N = 100
theta = 1
...
Plot(emp_dist(simulated_table_counts))
plt.xlabel('Number of Tables');

###  c) ###
Draw the empirical histogram of the number of tables in the case $N = 100$ and $\theta = 0.5$, based on 2000 repetitions. 

In [ ]:

...
plt.xlabel('Number of Tables');

### d) ###
Draw the empirical histogram of the number of tables in the case $N = 100$ and $\theta = 2$, based on 2000 repetitions. **Save the array of simulated values because you will need it in Part 5.** Give it a special name if you need to.

### e) ###
Are the empirical histograms above consistent with the your answers to Part 3(d)? 


**Type your answer here**

#newpage

## Part 5: Expectation and SD of the Number of Tables ##

### a) [ON PAPER]###
Fix a positive integer $N$ and suppose the process is run till $N$ people have been seated. Let $T_N$ be the number of tables. Find $E(T_N)$.

[It is helpful to note that the number of tables is equal to the number of people who started new tables.]

### b) [BACK TO CODE] ###
Use the code cell below to calculate your answer in Part (a) in the case $\theta = 2$ and $N = 100$ as well as the mean of the simulated values in Part 4(d).

Use the subsequent cell to compare the two resulting values.

In [ ]:

simulated_mean = ...

def expected_T_N(N, theta):
    ...
    return e_T_N

simulated_mean, expected_T_N(100, 2)


**Type your answer here**

### c) ###
The sum $\sum_{i=1}^k \frac{1}{i}$ grows slowly with $k$, and is roughly $\log(k)$ for large $k$. Set $\theta = 2$. On the same axes, plot as a function of $N$ in the range 100 to 5000:

- Your answer to Part (a).
- $\theta \log(N)$.

The code uses `plt.plot` to draw the plots. The required first two arguments are an array of the values on the horizontal axis and an array of the values on the vertical axis. Your job is to create `array_N`, `theta_log_N`, and `expected_values` appropriately.

In [ ]:

theta = 2

array_N = ...

theta_log_N = ...

expected_values = ...
...

plt.plot(n, theta_log_n, lw=2, label='$\\theta\log(N)$')
plt.plot(n, expected_values, lw=2, label='$E(T_N)$')
plt.xlabel('$N$')
plt.legend();

The graphs should justify the statement, "The number of tables grows like $\theta \log(N)$."

### d) [ON PAPER]###

For $N$ any positive integer and $\theta > 0$, find $SD(T_N)$. 

### e) ###
In the first cell below, calculate your answer in Part **d** in the case $\theta = 2$ and $N = 100$ as well as the SD of the simulated values in Part 4(d).

In the subsequent cell, compare the resulting values.

In [ ]:

...

simulated_sd, sd_T_N(100, 2)


**Type your answer here**

$T_n$ is said to have a [Poisson Binomial](https://en.wikipedia.org/wiki/Poisson_binomial_distribution) distribution. The shapes of the histograms should remind you of both the binomial and the Poisson histograms.

## Conclusion ##
What you have learned:
- The assumptions of a clustering model in which individuals aren't labeled by type ahead of time and the number of clusters is unknown
- How to simulate data under this model
- An empirical approximation to the distribution of the total number of clusters
- Exact values of the mean and SD of the number of clusters, and the rate at which the mean grows

In [ ]:
import gsExport
gsExport.generateSubmission("Lab10.ipynb")